# Experiment 005: Extended Optimization Run

Run the C++ optimizer with much higher parameters:
- Parameters: `-n 50000 -r 200` (vs current `-n 10000 -r 96`)
- Run 5 seeds for diversity
- Apply full pipeline: C++ optimizer → backward propagation → fix_direction → fractional translation

**Starting point:** 84.901044 (from exp_003)
**Target:** 80-82 (estimated)

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import os
import subprocess
import shutil
import time

getcontext().prec = 25
scale_factor = Decimal("1e18")

print("Libraries loaded")

In [ ]:
# Fast scoring functions
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def score_group_fast(xs, ys, degs):
    n = len(xs)
    if n == 0:
        return float('inf')
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + xs[i]
        py = TX * s + TY * c + ys[i]
        all_x.extend(px)
        all_y.extend(py)
    all_x, all_y = np.array(all_x), np.array(all_y)
    side = max(all_x.max() - all_x.min(), all_y.max() - all_y.min())
    return side * side / n

def strip_s(val):
    s = str(val)
    return float(s[1:] if s.startswith('s') else s)

def score_csv_fast(filepath):
    try:
        df = pd.read_csv(filepath)
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        df['x_val'] = df['x'].apply(strip_s)
        df['y_val'] = df['y'].apply(strip_s)
        df['deg_val'] = df['deg'].apply(strip_s)
        total = 0.0
        for n in range(1, 201):
            g = df[df['N'] == n]
            if len(g) == n:
                total += score_group_fast(g['x_val'].values, g['y_val'].values, g['deg_val'].values)
            else:
                return float('inf'), {}
        return total, {}
    except:
        return float('inf'), {}

print("Scoring functions defined")

In [ ]:
# Copy C++ optimizer and starting submission
os.chdir('/home/code/experiments/005_extended_optimization')
shutil.copy('/home/code/experiments/003_ensemble/tree_packer_v2', './tree_packer_v2')
shutil.copy('/home/code/experiments/003_ensemble/tree_packer_v2.cpp', './tree_packer_v2.cpp')
shutil.copy('/home/code/experiments/004_ensemble_fixed/submission.csv', './submission.csv')

start_score, _ = score_csv_fast('./submission.csv')
print(f"Starting score: {start_score:.6f}")

In [ ]:
# Run extended optimization with higher parameters
print("=" * 60)
print("PHASE 1: Extended C++ Optimization (5 seeds, -n 50000 -r 200)")
print("=" * 60)

best_score = start_score
best_file = './submission.csv'

for seed in range(5):
    print(f"\n--- Seed {seed} ---")
    
    # Copy current best to working file
    if seed > 0:
        shutil.copy(best_file, './submission.csv')
    
    start_time = time.time()
    result = subprocess.run(
        ['./tree_packer_v2', '-n', '50000', '-r', '200', '-s', str(seed)],
        capture_output=True, text=True,
        cwd='/home/code/experiments/005_extended_optimization',
        timeout=1200  # 20 min timeout per seed
    )
    elapsed = time.time() - start_time
    
    print(f"Completed in {elapsed:.1f}s")
    print(result.stdout[-500:] if len(result.stdout) > 500 else result.stdout)
    
    if os.path.exists('./submission_optimized.csv'):
        new_score, _ = score_csv_fast('./submission_optimized.csv')
        print(f"Score: {new_score:.6f}")
        
        if new_score < best_score:
            best_score = new_score
            shutil.copy('./submission_optimized.csv', f'./best_seed{seed}.csv')
            best_file = f'./best_seed{seed}.csv'
            shutil.copy(best_file, './submission.csv')
            print(f"NEW BEST: {best_score:.6f}")

print(f"\nPhase 1 best score: {best_score:.6f}")

In [ ]:
# Phase 2: Backward Propagation with multiple passes
print("\n" + "=" * 60)
print("PHASE 2: Aggressive Backward Propagation (3 passes)")
print("=" * 60)

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def load_trees(n, df):
    group = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_side(trees):
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    return max(xys.max(axis=0) - xys.min(axis=0))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        for idx in tree_index.query(poly):
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def backward_propagation_multi(input_file, output_file, passes=3):
    df = pd.read_csv(input_file)
    configs, sides = {}, {}
    for n in range(1, 201):
        trees = load_trees(n, df)
        if trees:
            configs[n] = trees
            sides[n] = get_side(trees)
    
    initial = sum(s**2/n for n, s in sides.items())
    print(f"Initial: {initial:.6f}")
    
    total_improvements = 0
    for p in range(passes):
        improvements = 0
        for n in range(200, 1, -1):
            if n not in configs or (n-1) not in configs:
                continue
            current_side = sides[n-1]
            best_side, best_idx = current_side, None
            for tree_idx in range(n):
                candidate = [t for i, t in enumerate(configs[n]) if i != tree_idx]
                if len(candidate) != n - 1:
                    continue
                cand_side = get_side(candidate)
                if cand_side < best_side and not has_overlap(candidate):
                    best_side, best_idx = cand_side, tree_idx
            if best_idx is not None:
                configs[n-1] = [t for i, t in enumerate(configs[n]) if i != best_idx]
                sides[n-1] = best_side
                improvements += 1
        total_improvements += improvements
        print(f"  Pass {p+1}: {improvements} improvements")
        if improvements == 0:
            break
    
    rows = []
    for n in range(1, 201):
        if n in configs:
            for i, tree in enumerate(configs[n]):
                rows.append({'id': f"{n:03d}_{i}", 'x': f"s{float(tree.center_x)}", 'y': f"s{float(tree.center_y)}", 'deg': f"s{float(tree.angle)}"})
    pd.DataFrame(rows).to_csv(output_file, index=False)
    final = sum(s**2/n for n, s in sides.items())
    print(f"Final: {final:.6f} (total improvements: {total_improvements})")
    return final

bp_score = backward_propagation_multi(best_file, './submission_bp.csv', passes=3)
if bp_score < best_score:
    best_score = bp_score
    best_file = './submission_bp.csv'
    print(f"Backward propagation improved to: {best_score:.6f}")

In [ ]:
# Phase 3: Fix Direction with fine sweep
print("\n" + "=" * 60)
print("PHASE 3: Fix Direction (Fine Rotation Sweep)")
print("=" * 60)

def calculate_bbox_at_angle(angle_deg, points):
    rad = np.radians(angle_deg)
    c, s = np.cos(rad), np.sin(rad)
    rot = points.dot(np.array([[c, s], [-s, c]]))
    return max(rot.max(axis=0) - rot.min(axis=0))

def optimize_rotation_fine(trees, step=0.1):
    pts = []
    for t in trees:
        pts.extend(list(t.polygon.exterior.coords))
    pts = np.array(pts) / float(scale_factor)
    try:
        hull = pts[ConvexHull(pts).vertices]
    except:
        return get_side(trees), 0.0
    
    best_side = calculate_bbox_at_angle(0, hull)
    best_angle = 0.0
    for angle in np.arange(0, 90, step):
        side = calculate_bbox_at_angle(angle, hull)
        if side < best_side - 1e-10:
            best_side = side
            best_angle = angle
    return best_side, best_angle

def apply_rotation(trees, angle):
    if not trees or abs(angle) < 1e-9:
        return trees
    bounds = [t.polygon.bounds for t in trees]
    center = np.array([(min(b[0] for b in bounds) + max(b[2] for b in bounds)) / 2,
                       (min(b[1] for b in bounds) + max(b[3] for b in bounds)) / 2]) / float(scale_factor)
    rad = np.radians(angle)
    c, s = np.cos(rad), np.sin(rad)
    rot = np.array([[c, -s], [s, c]])
    pts = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    rotated = (pts - center).dot(rot.T) + center
    return [ChristmasTree(str(rotated[i, 0]), str(rotated[i, 1]), str(float(trees[i].angle) + angle)) for i in range(len(trees))]

def fix_direction_fine(input_path, output_path):
    df = pd.read_csv(input_path)
    configs, sides = {}, {}
    for n in range(1, 201):
        trees = load_trees(n, df)
        if trees:
            configs[n] = trees
            sides[n] = get_side(trees)
    
    print(f"Initial: {sum(s**2/n for n, s in sides.items()):.6f}")
    improved = 0
    for n in range(1, 201):
        if n not in configs or len(configs[n]) < 2:
            continue
        try:
            best_side, best_angle = optimize_rotation_fine(configs[n], step=0.1)
            if abs(best_angle) > 0.001 and best_side < sides[n] - 1e-10:
                rotated = apply_rotation(configs[n], best_angle)
                if not has_overlap(rotated):
                    configs[n] = rotated
                    sides[n] = best_side
                    improved += 1
        except:
            pass
    
    print(f"Improved {improved} groups")
    rows = []
    for n in range(1, 201):
        if n in configs:
            for i, tree in enumerate(configs[n]):
                rows.append({'id': f"{n:03d}_{i}", 'x': f"s{float(tree.center_x)}", 'y': f"s{float(tree.center_y)}", 'deg': f"s{float(tree.angle)}"})
    pd.DataFrame(rows).to_csv(output_path, index=False)
    final = sum(s**2/n for n, s in sides.items())
    print(f"Final: {final:.6f}")
    return final

fd_score = fix_direction_fine(best_file, './submission_fd.csv')
if fd_score < best_score:
    best_score = fd_score
    best_file = './submission_fd.csv'
    print(f"Fix direction improved to: {best_score:.6f}")

In [ ]:
# Phase 4: Validate and save
print("\n" + "=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

# Validate for overlaps
df = pd.read_csv(best_file)
overlaps = []
for n in range(1, 201):
    trees = load_trees(n, df)
    if trees and has_overlap(trees):
        overlaps.append(n)

final_score, _ = score_csv_fast(best_file)
print(f"Final score: {final_score:.6f}")
print(f"Overlapping groups: {overlaps}")
print(f"Validation: {'PASSED' if len(overlaps) == 0 else 'FAILED'}")

if len(overlaps) == 0:
    shutil.copy(best_file, '/home/submission/submission.csv')
    print("Copied to /home/submission/submission.csv")

print(f"\n=== SUMMARY ===")
print(f"Starting: {start_score:.6f}")
print(f"Final: {final_score:.6f}")
print(f"Improvement: {start_score - final_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {final_score - 68.931058:.6f}")